# Pipple Lecture #12 - Reinforcement Learning
Now, you have seen quite some information relating to Reinforcement Learning. In this notebook, you will have the chance to program your own Deep Reinforcement Learning model. At least... tune its parameters. The programming of the game-environment, state-transitions, reward-calculations and training of the model has already been prepared for you. It is your job to focus on one task and one task only: keep your pole straight up!

During the lecture, we have not been able to discuss all elements of a DRL-model, as there are many aspects which can be tuned to perfection (or far from it). Some additional explanation will be given in the notebook where deemed necessary, but don't be shy to ask more!

## 1. Clone git-repo
Clone necessary data and install missing packages. This may take a few minutes, but will only have to be ran once.

In [ ]:
!git clone https://github.com/PippleNL/Lecture_RL.git
!pip install wandb
!pip install tensorflow==1.14

In [ ]:
import sys

root_path = '/content/Lecture_RL/keras-rl'
if root_path not in sys.path:
  sys.path.append(root_path)

## 1. Importing relevant modules
Let's get started. First, import necessary modules (and suppress some unwanted warnings). The 'gym' package is imported to be able to create a Cart Pole environment for you to play with. Further on, 'keras' enables the usage of a neural network, while 'keras-rl' contains a whole bunch of interesting Reinforcement Learning functions.

In [1]:
import numpy as np
import gym
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam

from rl.agents.dqn import DQNAgent
from rl.policy import EpsGreedyQPolicy
from rl.memory import SequentialMemory

Using TensorFlow backend.


## 2. Setting variables
Then, set the relevant variables. Get the environment and extract the number of actions available in the Cartpole problem. The seed settings can be useful to compare your results over different runs. However, both a neural network as the RL framework itself still contain a high level of randomization, which may make comparison of distinct runs difficult. Keep this in mind when trying different parameter settings

In [2]:
env = gym.make('CartPole-v0')
np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n

## 3. Set up your neural network
Next, build a neural network model. Initially, it is set to a simple feed-forward neural net, with a single hidden layer and 4 nodes. Try different settings by yourself, to find your optimal set-up! Unfortunately, until the day of today, there are no clear rules for choosing how many layers or nodes to use. Google may give you some idea, but most decisions still follow the famous method of trial-and-error.

Try tuning the number of hidden layers, the number of nodes per hidden layer, and the type of activation functions in the hidden and output layers. Use the 'print(model.summary())' to get an overview of the complexity of your model.

In [3]:
model = Sequential()
model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
model.add(Dense(4))
model.add(Activation('relu'))
#model.add(Dense(4))
#model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('linear'))
print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 4)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 20        
_________________________________________________________________
activation_1 (Activation)    (None, 4)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 10        
_________________________________________________________________
activation_2 (Activation)    (None, 2)                 0         
Total params: 30
Trainable params: 30
Non-trainable params: 0
_________________________________________________________________
None


## 4. Create your learning agent

Now, configure and compile your agent. The memory is set to Sequential Memory, storing the result of performed actions and obtained rewards. Try using different types of action-selection policies, memory sizes, learning rates, training steps, or w/e you can think of. Settings you can tune:

* **policy**: the way in which actions are selected over time, following some balancing method. This RL-concept is very important, incorporating a trade-off between exploring unknown parts of the environment, and exploiting known information. (possible policies: EpsGreedyQPolicy, LinearAnnealedPolicy, SoftmaxPolicy, GreedyQPolicy, BoltzmannQPolicy, MaxBoltzmannQPolicy, BoltzmannGumbelQPolicy)
* **memory limit**: the number of previous actions+rewards that are taken into account while learning, at a certain moment in time.
* **window_length**: actually not sure... just keep it at 1 to avoid errors ;)
* **target_model_update**: in theory denoted by $\alpha$, the network's learning rate. It determines how quickly the algorithm wants to converge to found target values (such as Q-values).

In [4]:
policy = EpsGreedyQPolicy()
memory = SequentialMemory(limit=10000, window_length=1)
dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, target_model_update=1e-2, policy=policy)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

## 5. The long process of learning...
Now it's time to learn something! There are four settings you can consider changing, however, only one which has an effect on your training performance:

* **nb_steps**: the larger, the more time your bot gets for trying to find a good strategy, but the longer you'll have to wait.
* **verbose**: printing running status. 0 for no logging, 1 for interval logging, 2 for episode logging
* **visualize**: you can visualize the training for show, but this mostly slows down training
* **log_interval**: if verbose=1, the number of steps that are considered to be an interval

In [9]:
dqn.fit(env, nb_steps=50000, verbose=1, visualize=False, log_interval=10)

Training for 50000 steps ...
Interval 1 (0 steps performed)
10/10 [==============================] - 0s 2ms/step - reward: 1.0000
Interval 2 (10 steps performed)
10/10 [==============================] - 0s 2ms/step - reward: 1.0000
Interval 3 (20 steps performed)
10/10 [==============================] - 0s 2ms/step - reward: 1.0000
Interval 4 (30 steps performed)
10/10 [==============================] - 0s 2ms/step - reward: 1.0000
Interval 5 (40 steps performed)
10/10 [==============================] - 0s 1ms/step - reward: 1.0000
Interval 6 (50 steps performed)
10/10 [==============================] - 0s 1ms/step - reward: 1.0000
Interval 7 (60 steps performed)
10/10 [==============================] - 0s 2ms/step - reward: 1.0000
Interval 8 (70 steps performed)
10/10 [==============================] - 0s 2ms/step - reward: 1.0000
Interval 9 (80 steps performed)
10/10 [==============================] - 0s 1ms/step - reward: 1.0000
Interval 10 (90 steps performed)
10/10 [==============

10/10 [==============================] - 0s 2ms/step - reward: 1.0000
Interval 78 (770 steps performed)
10/10 [==============================] - 0s 2ms/step - reward: 1.0000
1 episodes - episode_reward: 200.000 [200.000, 200.000]

Interval 79 (780 steps performed)
10/10 [==============================] - 0s 2ms/step - reward: 1.0000
Interval 80 (790 steps performed)
10/10 [==============================] - 0s 1ms/step - reward: 1.0000
Interval 81 (800 steps performed)
10/10 [==============================] - 0s 3ms/step - reward: 1.0000
Interval 82 (810 steps performed)
10/10 [==============================] - 0s 2ms/step - reward: 1.0000
Interval 83 (820 steps performed)
10/10 [==============================] - 0s 1ms/step - reward: 1.0000
Interval 84 (830 steps performed)
10/10 [==============================] - 0s 1ms/step - reward: 1.0000
Interval 85 (840 steps performed)
10/10 [==============================] - 0s 1ms/step - reward: 1.0000
Interval 86 (850 steps performed)
10/10 [

10/10 [==============================] - 0s 4ms/step - reward: 1.0000
Interval 147 (1460 steps performed)
10/10 [==============================] - 0s 4ms/step - reward: 1.0000
Interval 148 (1470 steps performed)
10/10 [==============================] - 0s 4ms/step - reward: 1.0000
Interval 149 (1480 steps performed)
10/10 [==============================] - 0s 4ms/step - reward: 1.0000
Interval 150 (1490 steps performed)
10/10 [==============================] - 0s 4ms/step - reward: 1.0000
1 episodes - episode_reward: 48.000 [48.000, 48.000] - loss: 0.483 - mean_absolute_error: 11.404 - mean_q: 22.781

Interval 151 (1500 steps performed)
10/10 [==============================] - 0s 4ms/step - reward: 1.0000
Interval 152 (1510 steps performed)
10/10 [==============================] - 0s 4ms/step - reward: 1.0000
Interval 153 (1520 steps performed)
10/10 [==============================] - 0s 4ms/step - reward: 1.0000
Interval 154 (1530 steps performed)
10/10 [==============================

10/10 [==============================] - 0s 4ms/step - reward: 1.0000
Interval 215 (2140 steps performed)
10/10 [==============================] - 0s 4ms/step - reward: 1.0000
Interval 216 (2150 steps performed)
10/10 [==============================] - 0s 4ms/step - reward: 1.0000
1 episodes - episode_reward: 63.000 [63.000, 63.000] - loss: 0.352 - mean_absolute_error: 12.123 - mean_q: 24.339

Interval 217 (2160 steps performed)
10/10 [==============================] - 0s 4ms/step - reward: 1.0000
Interval 218 (2170 steps performed)
10/10 [==============================] - 0s 3ms/step - reward: 1.0000
Interval 219 (2180 steps performed)
10/10 [==============================] - 0s 4ms/step - reward: 1.0000
Interval 220 (2190 steps performed)
10/10 [==============================] - 0s 4ms/step - reward: 1.0000
Interval 221 (2200 steps performed)
10/10 [==============================] - 0s 3ms/step - reward: 1.0000
Interval 222 (2210 steps performed)
10/10 [==============================

10/10 [==============================] - 0s 4ms/step - reward: 1.0000
Interval 282 (2810 steps performed)
10/10 [==============================] - 0s 5ms/step - reward: 1.0000
Interval 283 (2820 steps performed)
10/10 [==============================] - 0s 4ms/step - reward: 1.0000
Interval 284 (2830 steps performed)
10/10 [==============================] - 0s 4ms/step - reward: 1.0000
Interval 285 (2840 steps performed)
10/10 [==============================] - 0s 4ms/step - reward: 1.0000
Interval 286 (2850 steps performed)
10/10 [==============================] - 0s 4ms/step - reward: 1.0000
Interval 287 (2860 steps performed)
10/10 [==============================] - 0s 5ms/step - reward: 1.0000
1 episodes - episode_reward: 75.000 [75.000, 75.000] - loss: 0.395 - mean_absolute_error: 13.288 - mean_q: 26.713

Interval 288 (2870 steps performed)
10/10 [==============================] - 0s 4ms/step - reward: 1.0000
Interval 289 (2880 steps performed)
10/10 [==============================

...

10/10 [==============================] - 0s 4ms/step - reward: 1.0000
Interval 2914 (29130 steps performed)
10/10 [==============================] - 0s 4ms/step - reward: 1.0000
Interval 2915 (29140 steps performed)
10/10 [==============================] - 0s 3ms/step - reward: 1.0000
Interval 2916 (29150 steps performed)
10/10 [==============================] - 0s 3ms/step - reward: 1.0000
Interval 2917 (29160 steps performed)
10/10 [==============================] - 0s 4ms/step - reward: 1.0000
Interval 2918 (29170 steps performed)
10/10 [==============================] - 0s 4ms/step - reward: 1.0000
Interval 2919 (29180 steps performed)
10/10 [==============================] - 0s 3ms/step - reward: 1.0000
Interval 2920 (29190 steps performed)
10/10 [==============================] - 0s 3ms/step - reward: 1.0000
Interval 2921 (29200 steps performed)
10/10 [==============================] - 0s 4ms/step - reward: 1.0000
Interval 2922 (29210 steps performed)
10/10 [=====================

10/10 [==============================] - 0s 4ms/step - reward: 1.0000
Interval 2985 (29840 steps performed)
10/10 [==============================] - 0s 3ms/step - reward: 1.0000
Interval 2986 (29850 steps performed)
10/10 [==============================] - 0s 3ms/step - reward: 1.0000
Interval 2987 (29860 steps performed)
10/10 [==============================] - 0s 3ms/step - reward: 1.0000
Interval 2988 (29870 steps performed)
10/10 [==============================] - 0s 4ms/step - reward: 1.0000
Interval 2989 (29880 steps performed)
10/10 [==============================] - 0s 4ms/step - reward: 1.0000
Interval 2990 (29890 steps performed)
10/10 [==============================] - 0s 3ms/step - reward: 1.0000
Interval 2991 (29900 steps performed)
10/10 [==============================] - 0s 3ms/step - reward: 1.0000
Interval 2992 (29910 steps performed)
10/10 [==============================] - 0s 4ms/step - reward: 1.0000
Interval 2993 (29920 steps performed)
10/10 [=====================

10/10 [==============================] - 0s 4ms/step - reward: 1.0000
Interval 3056 (30550 steps performed)
10/10 [==============================] - 0s 3ms/step - reward: 1.0000
Interval 3057 (30560 steps performed)
10/10 [==============================] - 0s 4ms/step - reward: 1.0000
Interval 3058 (30570 steps performed)
10/10 [==============================] - 0s 3ms/step - reward: 1.0000
Interval 3059 (30580 steps performed)
10/10 [==============================] - 0s 4ms/step - reward: 1.0000
Interval 3060 (30590 steps performed)
10/10 [==============================] - 0s 3ms/step - reward: 1.0000
Interval 3061 (30600 steps performed)
10/10 [==============================] - 0s 3ms/step - reward: 1.0000
Interval 3062 (30610 steps performed)
10/10 [==============================] - 0s 4ms/step - reward: 1.0000
Interval 3063 (30620 steps performed)
10/10 [==============================] - 0s 3ms/step - reward: 1.0000
Interval 3064 (30630 steps performed)
10/10 [=====================

10/10 [==============================] - 0s 4ms/step - reward: 1.0000
Interval 3127 (31260 steps performed)
10/10 [==============================] - 0s 4ms/step - reward: 1.0000
Interval 3128 (31270 steps performed)
10/10 [==============================] - 0s 4ms/step - reward: 1.0000
Interval 3129 (31280 steps performed)
10/10 [==============================] - 0s 4ms/step - reward: 1.0000
Interval 3130 (31290 steps performed)
10/10 [==============================] - 0s 3ms/step - reward: 1.0000
Interval 3131 (31300 steps performed)
10/10 [==============================] - 0s 4ms/step - reward: 1.0000
Interval 3132 (31310 steps performed)
10/10 [==============================] - 0s 4ms/step - reward: 1.0000
Interval 3133 (31320 steps performed)
10/10 [==============================] - 0s 4ms/step - reward: 1.0000
1 episodes - episode_reward: 145.000 [145.000, 145.000] - loss: 0.958 - mean_absolute_error: 37.860 - mean_q: 75.632

Interval 3134 (31330 steps performed)
10/10 [===========

KeyError: 'TimeLimit.truncated'

## 6. How well do you perform?
Run the below code to test your DRL model. The larger the reward and number of steps per episode, the better your model performs. Running about 10 episodes will give you a proper overall status.

**NOTE:** Don't close the graph after/while running it. This will reset the kernel and cause you having to re-run everything. You can simply re-run the below code instead, each time.

In [10]:
dqn.test(env, nb_episodes=10, visualize=True)

Testing for 10 episodes ...
Episode 1: reward: 158.000, steps: 158
Episode 2: reward: 171.000, steps: 171
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 163.000, steps: 163
Episode 5: reward: 139.000, steps: 139
Episode 6: reward: 150.000, steps: 150
Episode 7: reward: 200.000, steps: 200
Episode 8: reward: 200.000, steps: 200
Episode 9: reward: 200.000, steps: 200
Episode 10: reward: 175.000, steps: 175
